Midi file player https://midiplayer.ehubsoft.net/

# Setup

In [1]:
!curl -L https://cseweb.ucsd.edu/classes/sp25/cse253-a/data/student_files.tar.gz -o student_files.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1272M  100 1272M    0     0  7569k      0  0:02:52  0:02:52 --:--:--  9.8M


In [ ]:
# !rm student_files.tar.gz

In [2]:
# Make sure to wait until the file fully finishes uploading
# !rm -rf student_files
!tar -xvzf student_files.tar.gz | tail -n 5  # Make sure there's no error message at the tail

student_files/task1_composer_classification/midis/1022.mid
student_files/task1_composer_classification/midis/147.mid
student_files/task1_composer_classification/midis/275.mid
student_files/task1_composer_classification/midis/765.mid
student_files/task1_composer_classification/midis/701.mid


In [3]:
!pip install librosa | tail -n 1  # I don't want a super long output
!pip install miditoolkit | tail -n 1  # I don't want a super long output
!pip install xgboost | tail -n 1  # I don't want a super long output
!pip install lightgbm | tail -n 1  # I don't want a super long output
!pip install autogluon | tail -n 1  # I don't want a super long output

In [4]:
# Probably more imports than are really necessary...
import os
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
from tqdm import tqdm
import librosa
import numpy as np
import miditoolkit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, average_precision_score, accuracy_score
import random

from mido import MidiFile
from sklearn.model_selection import train_test_split
from music21 import converter, chord, stream
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd

# Features that didn't work

In [ ]:
# def get_chord_features(midi_path):
#     try:
#         midi_stream = converter.parse(midi_path)
#         chords = list(midi_stream.chordify().getElementsByClass(chord.Chord))

#         if not chords:
#             print("no chords")
#             return [], []

#         chord_roots = [c.root().midi % 12 for c in chords]  # Pitch class of the root
#         chord_qualities = [c.quality for c in chords]
#         num_unique_pitches = [len(c.pitches) for c in chords]
#         is_major_triad = [1 if c.isMajorTriad() else 0 for c in chords]
#         is_minor_triad = [1 if c.isMinorTriad() else 0 for c in chords]
#         is_dominant_7th = [1 if c.isDominantSeventh() else 0 for c in chords]

#         # Basic chord progression (root note transitions)
#         root_note_transitions = []
#         for i in range(len(chord_roots) - 1):
#             root_note_transitions.append((chord_roots[i], chord_roots[i+1]))

#         return [chord_roots, chord_qualities, num_unique_pitches,
#                 is_major_triad, is_minor_triad, is_dominant_7th,
#                 root_note_transitions]

#     except Exception as e:
#         print(f"Error processing {midi_path} for chords (music21): {e}")
#         return []

In [ ]:
# def features(path):
#     midi_obj = miditoolkit.midi.parser.MidiFile(dataroot1 + '/' + path)

#     notes = midi_obj.instruments[0].notes

    # Key (2 features) - DIDN'T WORK
    # key_info = midi_to_audio_features(dataroot1 + path)
    # if key_info:
    #     tonic = key_info[0] # Estimated tonic (e.g., 'C', 'C#', 'D', ...)
    #     mode = "major" if key_info[1] > 0 else "minor" # Estimated mode (simplified)
    #     # You might want to map tonic to a numerical representation
    #     tonic_val = librosa.note_to_midi(tonic + '0') % 12 # Get pitch class (0-11)
    #     features.extend([tonic_val, 1 if mode == "major" else 0]) # Add as numerical features
    # else:
    #     features.extend([-1, -1]) # Indicate failure to detect key

    # Chords (7 features) - DIDN'T WORK
    # features.extend(get_chord_features(dataroot1 + path))

# Getting data

In [5]:
dataroot1 = "student_files/task1_composer_classification/"

In [24]:
def features(path):
    midi_obj = miditoolkit.midi.parser.MidiFile(dataroot1 + '/' + path)

    notes = midi_obj.instruments[0].notes

    # baseline (2 features)
    num_notes = len(notes)
    average_pitch = sum([note.pitch for note in notes]) / num_notes
    average_duration = sum([note.end - note.start for note in notes]) / num_notes
    features = [average_pitch, average_duration]

    # notes durations and density (4 features)
    durations = [note.end - note.start for note in notes]
    # std_dev_duration = np.std(durations)
    short_threshold = 0.5 * np.mean(durations)  # or use a fixed value like 0.25
    short_notes = [d for d in durations if d < short_threshold]
    short_note_ratio = len(short_notes) / len(durations)
    mid = MidiFile(dataroot1 + path)
    # total_time = mid.length  # in seconds
    note_density = num_notes / mid.length
    features.extend([short_note_ratio, note_density])

    # Volume (3 features)
    velocities = np.array([note.velocity for note in notes])
    # average_velocity = np.mean(velocities)
    # median_velocity = np.median(velocities)
    std_dev_velocity = np.std(velocities)
    velocity_range = np.max(velocities) - np.min(velocities)
    features.extend([std_dev_velocity, velocity_range])

    # Get tempo changes (3 features)
    tempo_changes = midi_obj.tempo_changes
    tempo_values = []
    tempo_times = []
    average_tempo = 120 # Default tempo if no tempo changes
    std_dev_tempo = 0
    # num_tempo_changes = 0
    if tempo_changes:
        # Sort tempo changes by time
        tempo_changes.sort(key=lambda x: x.time)
        tempo_values = np.array([tempo.tempo for tempo in tempo_changes])
        tempo_times = np.array([tempo.time for tempo in tempo_changes])

        average_tempo = np.mean(tempo_values)
        std_dev_tempo = np.std(tempo_values)
        # num_tempo_changes = len(tempo_values)
    features.extend([average_tempo, std_dev_tempo])

    # Register (5 features)
    pitches = np.array([note.pitch for note in notes])
    pitch_range = np.max(pitches) - np.min(pitches) if pitches.size > 0 else 0
    pitch_max = np.max(pitches)
    high_notes = [note for note in notes if note.pitch > 72]  # use 72 (C5, one octave above middle C) as a threshold
    high_register_ratio = len(high_notes) / len(notes)
    pitches_std_dev = np.std(pitches)
    # pitch_mad = np.mean([abs(p - average_pitch) for p in pitches])
    features.extend([pitch_range, pitches_std_dev, pitch_max, high_register_ratio])

    # Left and right hands (2 features)
    # Standard deviation of the highest and lowest 35% of notes (as a proxy for right hand and left hand melodies)
    # Idk how accurate this is but let's try, I feel like if std dev is lower then it might be like Mozart and
    # if it's higher it might be Chopin or smth
    percentile = 35
    sorted_pitches = np.sort(pitches)
    num_highest_and_lowest_notes = int(np.ceil(num_notes * (percentile / 100)))
    highest_pitches = sorted_pitches[-num_highest_and_lowest_notes:]
    lowest_pitches = sorted_pitches[:num_highest_and_lowest_notes]
    # highest_notes_std_dev = np.std(highest_pitches)
    # lowest_notes_std_dev = np.std(lowest_pitches)
    highest_notes_range = np.max(highest_pitches) - np.min(highest_pitches)
    lowest_notes_range = np.max(lowest_pitches) - np.min(lowest_pitches)
    features.extend([highest_notes_range, lowest_notes_range])

    # melodic pitch jumps (2 features)
    # notes_sorted = sorted(notes, key=lambda n: (n.start, -n.pitch))  # -pitch to get highest first
    time_to_highest = {}  # get highest note at each time step
    for note in notes:
        if note.start not in time_to_highest or note.pitch > time_to_highest[note.start]:
            time_to_highest[note.start] = note.pitch
    melody_pitches = [time_to_highest[t] for t in sorted(time_to_highest)]
    pitch_jumps = [abs(melody_pitches[i+1] - melody_pitches[i]) for i in range(len(melody_pitches) - 1)]
    avg_jump = np.mean(pitch_jumps)
    # std_jump = np.std(pitch_jumps)
    # max_jump = np.max(pitch_jumps)
    features.extend([avg_jump])

    # Pitch classes
    pitch_class_counts = [0] * 12
    for note in notes:
        pc = note.pitch % 12
        pitch_class_counts[pc] += 1
    pitch_class_distribution = [count / len(notes) for count in pitch_class_counts]
    features.extend(pitch_class_distribution)

    # direction changes (1 feature)
    diffs = [melody_pitches[i+1] - melody_pitches[i] for i in range(len(melody_pitches) - 1)]
    directions = [np.sign(d) for d in diffs]
    direction_changes = sum(
        1 for i in range(len(directions) - 1) if directions[i] != 0 and directions[i] != directions[i+1]
    )
    direction_changes_ratio = direction_changes / (len(directions) - 1) if len(directions) > 1 else 0.0
    features.extend([direction_changes_ratio])

    # Time signature (1 feature)
    tsig_count = len(midi_obj.time_signature_changes)
    features.extend([tsig_count])

    # Autocorrelation
    velocities = [note.velocity for note in notes]
    v = np.array(velocities)
    v_mean = np.mean(v)
    numerator = np.sum((v[:-1] - v_mean) * (v[1:] - v_mean))
    denominator = np.sum((v - v_mean) ** 2)
    vel_autocorr_1 = numerator / denominator if denominator != 0 else 0.0
    features.extend([vel_autocorr_1])

    # IOI (inter-onset interval)
    onsets = sorted([note.start for note in notes])
    ioi = np.diff(onsets)  # list of time differences between consecutive onsets
    ioi_mean = np.mean(ioi)
    ioi_std = np.std(ioi)
    features.extend([ioi_mean, ioi_std])

    # Unique pitch ratio
    unique_pitches = len(set(pitches))
    unique_pitch_ratio = unique_pitches / len(pitches)
    features.extend([unique_pitch_ratio])

    # octave bins
    bins = np.arange(0, 128, 12)  # bins edges: 0, 12, 24, ..., 120
    octave_hist, _ = np.histogram(pitches, bins=bins)
    octave_hist = octave_hist / octave_hist.sum()  # normalize
    features.extend(octave_hist.tolist())

    # Rhythmic Entropy
    if len(ioi) > 1:
        # Bin IOIs (rounding helps reduce noise and collapse near-duplicates)
        binned_ioi = np.round(ioi, decimals=3)  # Adjust bin size as needed
        values, counts = np.unique(binned_ioi, return_counts=True)
        probs = counts / counts.sum()
        rhythmic_entropy = -np.sum(probs * np.log2(probs))
    else:
        rhythmic_entropy = 0.0  # No rhythm to analyze
    features.extend([rhythmic_entropy])

    return features

In [25]:
# Training data
train_path = dataroot1 + "/train.json"

with open(train_path, 'r') as f:
    train_json = eval(f.read())

files_train_all = [k for k in train_json]
X_train_all = [features(k) for k in train_json]
y_train_all = [train_json[k] for k in train_json]

In [26]:
X_train, X_val, y_train, y_val, files_train, files_val = train_test_split(
        X_train_all, y_train_all, files_train_all, test_size=0.2, shuffle=True, random_state=42
    )

In [27]:
print(files_train_all[5])
print(X_train_all[5])
print(y_train_all[5])

midis/5.mid
[66.50220264317181, 344.8898678414097, 0.7577092511013216, 9.993379343078677, np.float64(20.247227039731246), np.int64(95), np.float64(167.99993280002687), np.float64(0.0), np.int64(56), np.float64(11.398034628226418), np.int64(91), 0.2643171806167401, np.int64(20), np.int64(29), np.float64(5.829545454545454), 0.07048458149779736, 0.02643171806167401, 0.05286343612334802, 0.09691629955947137, 0.048458149779735685, 0.05286343612334802, 0.07048458149779736, 0.2775330396475771, 0.05726872246696035, 0.030837004405286344, 0.013215859030837005, 0.2026431718061674, 0.6742857142857143, 1, np.float64(0.953737582136841), np.float64(134.44247787610618), np.float64(211.74159564991805), 0.19383259911894274, 0.0, 0.0, 0.004405286343612335, 0.08370044052863436, 0.15859030837004406, 0.45374449339207046, 0.24669603524229075, 0.05286343612334802, 0.0, 0.0, np.float64(2.3246125989162727)]
Beethoven


In [28]:
print(files_train[5])
print(X_train[5])
print(y_train[5])

midis/872.mid
[62.39460370994941, 139.858347386172, 0.0893760539629005, 18.60452955978962, np.float64(30.243173816414657), np.int64(94), np.float64(120.0), np.float64(0.0), np.int64(73), np.float64(15.062750294289366), np.int64(102), 0.2411467116357504, np.int64(36), np.int64(27), np.float64(13.12797619047619), 0.09949409780775717, 0.09274873524451939, 0.05902192242833052, 0.0893760539629005, 0.045531197301854974, 0.1045531197301855, 0.1551433389544688, 0.01517706576728499, 0.09612141652613827, 0.1433389544688027, 0.045531197301854974, 0.05396290050590219, 0.5850746268656717, 1, np.float64(0.9860748786709461), np.float64(51.55236486486486), np.float64(50.01365314755733), 0.11804384485666104, 0.0, 0.0, 0.03372681281618887, 0.11298482293423272, 0.3102866779089376, 0.2866779089376054, 0.1315345699831366, 0.11298482293423272, 0.011804384485666104, 0.0, np.float64(1.596662401863128)]
Liszt


In [29]:
print(files_val[5])
print(X_val[5])
print(y_val[5])

midis/943.mid
[56.666666666666664, 266.9375, 0.17083333333333334, 20.787422362225513, np.float64(18.23364516415251), np.int64(65), np.float64(330.00033000033), np.float64(0.0), np.int64(60), np.float64(18.953598309790383), np.int64(89), 0.25416666666666665, np.int64(27), np.int64(18), np.float64(11.151515151515152), 0.08333333333333333, 0.06666666666666667, 0.125, 0.09583333333333334, 0.058333333333333334, 0.12083333333333333, 0.020833333333333332, 0.32083333333333336, 0.05416666666666667, 0.0, 0.025, 0.029166666666666667, 0.7404580152671756, 1, np.float64(0.8955237093785035), np.float64(126.52719665271967), np.float64(128.55543670482078), 0.175, 0.0, 0.0, 0.1625, 0.19583333333333333, 0.25833333333333336, 0.12083333333333333, 0.06666666666666667, 0.19583333333333333, 0.0, 0.0, np.float64(1.4958613224704465)]
Beethoven


In [30]:
groundtruth_train_all = {k: train_json[k] for k in train_json}
groundtruth_train = {k: train_json[k] for k in files_train}
groundtruth_val = {k: train_json[k] for k in files_val}

In [31]:
# Testing data
test_path = dataroot1 + "/test.json"

d = eval(open(test_path, 'r').read())

files_test = []
X_test = []
for k in d:
    files_test.append(k)
    X_test.append(features(k))

In [32]:
print(files_test[5])
print(X_test[5])

midis/1215.mid
[61.4989247311828, 165.19569892473118, 0.002150537634408602, 16.109723796479532, np.float64(22.173068820590668), np.int64(63), np.float64(132.000132000132), np.float64(0.0), np.int64(57), np.float64(9.592587804095453), np.int64(88), 0.1032258064516129, np.int64(23), np.int64(28), np.float64(5.660869565217391), 0.06236559139784946, 0.025806451612903226, 0.17204301075268819, 0.01935483870967742, 0.10752688172043011, 0.023655913978494623, 0.1032258064516129, 0.17204301075268819, 0.012903225806451613, 0.11827956989247312, 0.015053763440860216, 0.16774193548387098, 0.8995633187772926, 1, np.float64(0.9760917482174188), np.float64(65.17241379310344), np.float64(70.27633582460189), 0.09462365591397849, 0.0, 0.0, 0.002150537634408602, 0.08387096774193549, 0.3225806451612903, 0.47956989247311826, 0.0989247311827957, 0.012903225806451613, 0.0, 0.0, np.float64(1.2255148283907524)]


# Eval Function

In [33]:
def accuracy1(groundtruth: dict, predictions):
    correct = 0
    for k in groundtruth:
        if not (k in predictions):
            print("Missing " + str(k) + " from predictions")
            return 0
        if predictions[k] == groundtruth[k]:
            correct += 1
    print(correct, len(groundtruth), correct / len(groundtruth))
    return correct / len(groundtruth)

# Model: Logistic Regression

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [16]:
class logisticRegression():
    def __init__(self):
        pass

    # def predict(self, path, outpath=None):
    def predict(self, files, features, outpath=None):
        predictions = {}
        for i, x in enumerate(features):
            k = files[i]
            pred = self.model.predict([x])
            predictions[k] = str(pred[0])
            # print(k, pred)
        if outpath:
            with open(outpath, "w") as z:
                z.write(str(predictions) + '\n')
        return predictions

    # def train(self, path):
    def train(self, X_train, y_train):
        model = LogisticRegression(max_iter=5000, random_state=42)
        model.fit(X_train, y_train)
        self.model = model

In [17]:
def runLogisticRegressionWithVal():
    model = logisticRegression()
    model.train(X_train, y_train)
    train_preds = model.predict(files_train, X_train)
    val_preds = model.predict(files_val, X_val)
    test_preds = model.predict(files_test, X_test, "predictions1_train.json")

    # train_labels = eval(open(dataroot1 + "/train.json").read())
    # acc1 = accuracy1(train_labels, train_preds)
    acc1_train = accuracy1(groundtruth_train, train_preds)
    acc1_val = accuracy1(groundtruth_val, val_preds)
    print("Task 1 training accuracy = " + str(acc1_train))
    print("Task 1 validation accuracy = " + str(acc1_val))

In [18]:
!rm predictions1_train.json
runLogisticRegressionWithVal()

rm: cannot remove 'predictions1_train.json': No such file or directory


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


711 968 0.734504132231405
158 242 0.6528925619834711
Task 1 training accuracy = 0.734504132231405
Task 1 validation accuracy = 0.6528925619834711


In [19]:
# Train will all training data
def runLogisticRegression():
    model = logisticRegression()
    model.train(X_train_all, y_train_all)
    train_preds = model.predict(files_train_all, X_train_all)
    test_preds = model.predict(files_test, X_test, "predictions1.json")

    acc1 = accuracy1(groundtruth_train_all, train_preds)
    print("Task 1 training accuracy = " + str(acc1))

In [20]:
!rm predictions1.json
runLogisticRegression()

rm: cannot remove 'predictions1.json': No such file or directory


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


886 1210 0.7322314049586777
Task 1 training accuracy = 0.7322314049586777


# Model: XGBoost

https://xgboost.readthedocs.io/en/release_3.0.0/parameter.html

In [34]:
class XGBoost():
    def __init__(self):
        pass

    def predict(self, files, features, outpath=None):
        predictions = {}
        for i, x in enumerate(features):
            k = files[i]
            pred = self.model.predict([x])
            pred = self.label_encoder.inverse_transform(pred)
            predictions[k] = str(pred[0])
        if outpath:
            with open(outpath, "w") as z:
                z.write(str(predictions) + '\n')
        return predictions

    def train(self, X_train, y_train):
        # XGBoost wants outputs in [0 1 2 3 4 5 6 7] instead of
        # ['Bach' 'Beethoven' 'Chopin' 'Haydn' 'Liszt' 'Mozart' 'Schubert' 'Schumann']
        # So we need to encode string labels to numerical values
        self.label_encoder = LabelEncoder()
        y_train_encoded = self.label_encoder.fit_transform(y_train)
        print(f"Classes found by LabelEncoder (should be 8): {self.label_encoder.classes_}")
        # initialize and train XGBoost classifier
        model = XGBClassifier(n_estimators=100, max_depth=10, learning_rate=0.1, objective='multi:softmax')
        model.fit(X_train, y_train_encoded)
        self.model = model

In [35]:
def runXGBoost():
    bst = XGBoost()
    bst.train(X_train, y_train)
    train_preds = bst.predict(files_train, X_train)
    val_preds = bst.predict(files_val, X_val)
    test_preds = bst.predict(files_test, X_test, "predictions1.json")

    acc1_train = accuracy1(groundtruth_train, train_preds)
    acc1_val = accuracy1(groundtruth_val, val_preds)
    print("Task 1 training accuracy = " + str(acc1_train))
    print("Task 1 validation accuracy = " + str(acc1_val))

In [36]:
!rm predictions1.json
runXGBoost()

Classes found by LabelEncoder (should be 8): ['Bach' 'Beethoven' 'Chopin' 'Haydn' 'Liszt' 'Mozart' 'Schubert'
 'Schumann']
968 968 1.0
199 242 0.8223140495867769
Task 1 training accuracy = 1.0
Task 1 validation accuracy = 0.8223140495867769


# Model: GradientBoostingClassifier

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

In [ ]:
class skLearnClassifier():
    def __init__(self):
        pass

    def predict(self, files, features, outpath=None):
        predictions = {}
        for i, x in enumerate(features):
            k = files[i]
            pred = self.model.predict([x])
            predictions[k] = str(pred[0])
        if outpath:
            with open(outpath, "w") as z:
                z.write(str(predictions) + '\n')
        return predictions

    def train(self, X_train, y_train):
        model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=2)
        model.fit(X_train, y_train)
        self.model = model

In [ ]:
def runSKLearnClassifier():
    gbc = skLearnClassifier()
    gbc.train(X_train, y_train)
    train_preds = gbc.predict(files_train, X_train)
    val_preds = gbc.predict(files_val, X_val)
    test_preds = gbc.predict(files_test, X_test, "predictions1.json")

    acc1_train = accuracy1(groundtruth_train, train_preds)
    acc1_val = accuracy1(groundtruth_val, val_preds)
    print("Task 1 training accuracy = " + str(acc1_train))
    print("Task 1 validation accuracy = " + str(acc1_val))

In [ ]:
!rm predictions1.json
runSKLearnClassifier()

# Model: LightGBM

https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html

In [ ]:
class lightGBM():
    def __init__(self):
        pass

    def predict(self, files, features, outpath=None):
        predictions = {}
        for i, x in enumerate(features):
            k = files[i]
            pred = self.model.predict([x])
            predictions[k] = str(pred[0])
        if outpath:
            with open(outpath, "w") as z:
                z.write(str(predictions) + '\n')
        return predictions

    def train(self, X_train, y_train):
        model = LGBMClassifier(num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, random_state=42)
        model.fit(X_train, y_train)
        self.model = model

In [ ]:
def runLightGBM():
    lgbm = lightGBM()
    lgbm.train(X_train, y_train)
    train_preds = lgbm.predict(files_train, X_train)
    val_preds = lgbm.predict(files_val, X_val)
    test_preds = lgbm.predict(files_test, X_test, "predictions1.json")

    acc1_train = accuracy1(groundtruth_train, train_preds)
    acc1_val = accuracy1(groundtruth_val, val_preds)
    print("Task 1 training accuracy = " + str(acc1_train))
    print("Task 1 validation accuracy = " + str(acc1_val))

In [ ]:
!rm predictions1.json
runLightGBM()

# Model: Autogluon

In [ ]:
!rm -rf AutogluonModels

In [ ]:
import multiprocessing
num_cores = multiprocessing.cpu_count()

In [ ]:
# hyperparameters={'GBM': {}, 'RF': {}}
hyperparams = {
    'XGB': {},         # XGBoost
    'GBM': {},         # LightGBM (AutoGluon's GBM refers to LightGBM)
    'CAT': {},         # CatBoost
    'RF': {},          # Random Forest
    # 'XT': {},          # Extra Trees
    'GBC': {}          # sklearn.ensemble.GradientBoostingClassifier
}
# someone said on discord: i can get higher acc with high_quality than best

In [ ]:
class AutoGluon():
    def __init__(self):
        self.label = 'label'

    def predict(self, files, features, outpath=None):
        df = pd.DataFrame(features)
        preds = self.model.predict(df)
        predictions = {}
        for i, k in enumerate(files):
            predictions[k] = str(preds.iloc[i])
        if outpath:
            with open(outpath, "w") as z:
                z.write(str(predictions) + '\n')
        return predictions

    def train(self, X_train, y_train):
        df_train = pd.DataFrame(X_train)
        df_train[self.label] = y_train
        # also try best_quality if there's time
        self.model = TabularPredictor(label=self.label).fit(
            df_train,
            presets='high_quality',
            num_cpus=num_cores,
            ag_args_fit={'num_gpus': 1},
            # verbosity=4,
            ag_args_ensemble =dict(fold_fitting_strategy='sequential_local'),  # disable Ray
            save_bag_folds=True
        )  # , num_cpus=num_cores

In [ ]:
# def runAutoGluon():
#     ag = AutoGluon()
#     ag.train(X_train_all, y_train_all)
#     train_preds = ag.predict(files_train_all, X_train_all)
#     val_preds = ag.predict(files_val, X_val)
#     test_preds = ag.predict(files_test, X_test, "predictions1.json")

#     acc1_train = accuracy1(groundtruth_train_all, train_preds_all)
#     acc1_val = accuracy1(groundtruth_val, val_preds)
#     print("Task 1 training accuracy = " + str(acc1_train))
#     print("Task 1 validation accuracy = " + str(acc1_val))

In [ ]:
# !rm predictions3.json
# runAutoGluon()

In [ ]:
!rm -rf AutogluonModels/

# ag = AutoGluon()
# ag.train(X_train_all, y_train_all)

# Try with a subset of the training data. Idk why but it gave me higher accuracies when I was using XGBoost etc
ag = AutoGluon()
ag.train(X_train, y_train)

In [ ]:
# df_train = pd.DataFrame(X_train_all)
# df_train[ag.label] = y_train_all
# ag.model.leaderboard(df_train)

df_train = pd.DataFrame(X_train)
df_train[ag.label] = y_train
ag.model.leaderboard(df_train)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist_BAG_L1_FULL,1.000000,NaN,accuracy,0.011162,0.016804,0.009907,0.011162,0.016804,0.009907,1,True,48
1,KNeighborsDist_BAG_L1,1.000000,0.680165,accuracy,0.014780,0.016804,0.009907,0.014780,0.016804,0.009907,1,True,2
2,NeuralNetTorch_BAG_L1_FULL,1.000000,NaN,accuracy,0.027998,NaN,6.931780,0.027998,NaN,6.931780,1,True,58
3,RandomForestEntr_BAG_L1_FULL,1.000000,NaN,accuracy,0.114830,0.153317,2.654643,0.114830,0.153317,2.654643,1,True,53
4,ExtraTrees_r42_BAG_L1,1.000000,0.805785,accuracy,0.115427,0.153354,1.937892,0.115427,0.153354,1.937892,1,True,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,CatBoost_r137_BAG_L1,0.853719,0.782645,accuracy,0.031458,0.025176,19.201638,0.031458,0.025176,19.201638,1,True,23
88,CatBoost_r13_BAG_L1_FULL,0.845455,NaN,accuracy,0.009927,NaN,2.106662,0.009927,NaN,2.106662,1,True,71
89,NeuralNetTorch_r86_BAG_L1_FULL,0.826446,NaN,accuracy,0.028210,NaN,0.687499,0.028210,NaN,0.687499,1,True,78
90,KNeighborsUnif_BAG_L1_FULL,0.769421,NaN,accuracy,0.012442,0.101894,0.010540,0.012442,0.101894,0.010540,1,True,47


In [ ]:
!rm predictions1.json

# train_preds_all = ag.predict(files_train_all, X_train_all)
train_preds = ag.predict(files_train, X_train)
val_preds = ag.predict(files_val, X_val)
test_preds = ag.predict(files_test, X_test, "predictions1.json")

# acc1_train = accuracy1(groundtruth_train_all, train_preds_all)
acc1_train = accuracy1(groundtruth_train, train_preds)
acc1_val = accuracy1(groundtruth_val, val_preds)
print("Task 3 training accuracy = " + str(acc1_train))
print("Task 3 validation accuracy = " + str(acc1_val))

1210 1210 1.0
242 242 1.0
Task 3 training accuracy = 1.0
Task 3 validation accuracy = 1.0


### Try predicting with a few other top models

In [ ]:
# https://auto.gluon.ai/0.3.1/api/autogluon.predictor.html
# predictor.predict(test_data, model=MODEL_NAME)

def predictAG(files, features, model_name, outpath=None):
    df = pd.DataFrame(features)
    preds = ag.model.predict(df)
    predictions = {}
    for i, k in enumerate(files):
        predictions[k] = str(preds.iloc[i])
    if outpath:
        with open(outpath, "w") as z:
            z.write(str(predictions) + '\n')
    return predictions

In [ ]:
!rm predictions1.json
model_name = 'WeightedEnsemble_L3'

train_preds_all = predictAG(files_train_all, X_train_all, model_name)
val_preds = predictAG(files_val, X_val, model_name)
test_preds = predictAG(files_test, X_test, model_name, "predictions1.json")

acc1_train = accuracy1(groundtruth_train_all, train_preds_all)
acc1_val = accuracy1(groundtruth_val, val_preds)
print("Task 3 training accuracy = " + str(acc1_train))
print("Task 3 validation accuracy = " + str(acc1_val))

1210 1210 1.0
242 242 1.0
Task 3 training accuracy = 1.0
Task 3 validation accuracy = 1.0


In [ ]:
!rm predictions1.json
model_name = 'KNeighborsDist_BAG_L1_FULL'

train_preds_all = predictAG(files_train_all, X_train_all, model_name)
val_preds = predictAG(files_val, X_val, model_name)
test_preds = predictAG(files_test, X_test, model_name, "predictions1.json")

acc1_train = accuracy1(groundtruth_train_all, train_preds_all)
acc1_val = accuracy1(groundtruth_val, val_preds)
print("Task 3 training accuracy = " + str(acc1_train))
print("Task 3 validation accuracy = " + str(acc1_val))

1210 1210 1.0
242 242 1.0
Task 3 training accuracy = 1.0
Task 3 validation accuracy = 1.0


In [ ]:
!rm predictions1.json
model_name = 'KNeighborsDist_BAG_L1'

train_preds_all = predictAG(files_train_all, X_train_all, model_name)
val_preds = predictAG(files_val, X_val, model_name)
test_preds = predictAG(files_test, X_test, model_name, "predictions1.json")

acc1_train = accuracy1(groundtruth_train_all, train_preds_all)
acc1_val = accuracy1(groundtruth_val, val_preds)
print("Task 3 training accuracy = " + str(acc1_train))
print("Task 3 validation accuracy = " + str(acc1_val))

1210 1210 1.0
242 242 1.0
Task 3 training accuracy = 1.0
Task 3 validation accuracy = 1.0


In [ ]:
!rm predictions1.json
model_name = 'NeuralNetTorch_BAG_L1_FULL'

train_preds_all = predictAG(files_train_all, X_train_all, model_name)
val_preds = predictAG(files_val, X_val, model_name)
test_preds = predictAG(files_test, X_test, model_name, "predictions1.json")

acc1_train = accuracy1(groundtruth_train_all, train_preds_all)
acc1_val = accuracy1(groundtruth_val, val_preds)
print("Task 3 training accuracy = " + str(acc1_train))
print("Task 3 validation accuracy = " + str(acc1_val))

1210 1210 1.0
242 242 1.0
Task 3 training accuracy = 1.0
Task 3 validation accuracy = 1.0


In [ ]:
!rm predictions1.json
model_name = 'RandomForestEntr_BAG_L1_FULL'

train_preds_all = predictAG(files_train_all, X_train_all, model_name)
val_preds = predictAG(files_val, X_val, model_name)
test_preds = predictAG(files_test, X_test, model_name, "predictions1.json")

acc1_train = accuracy1(groundtruth_train_all, train_preds_all)
acc1_val = accuracy1(groundtruth_val, val_preds)
print("Task 3 training accuracy = " + str(acc1_train))
print("Task 3 validation accuracy = " + str(acc1_val))

1210 1210 1.0
242 242 1.0
Task 3 training accuracy = 1.0
Task 3 validation accuracy = 1.0


In [ ]:
!rm predictions1.json
model_name = 'ExtraTrees_r42_BAG_L1'

train_preds_all = predictAG(files_train_all, X_train_all, model_name)
val_preds = predictAG(files_val, X_val, model_name)
test_preds = predictAG(files_test, X_test, model_name, "predictions1.json")

acc1_train = accuracy1(groundtruth_train_all, train_preds_all)
acc1_val = accuracy1(groundtruth_val, val_preds)
print("Task 3 training accuracy = " + str(acc1_train))
print("Task 3 validation accuracy = " + str(acc1_val))

1210 1210 1.0
242 242 1.0
Task 3 training accuracy = 1.0
Task 3 validation accuracy = 1.0


# Model: RF

# Model: tabpfn

# Model: SVM

# Model: CatBoost

# Model: ExtraTrees